In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/tic_tac_toe/tictactoemoves.csv')

In [3]:
df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,player,move
0,1,1,1,1,0,0,1,0,0,-1,-1
1,1,1,1,1,0,0,0,1,0,-1,-1
2,1,1,1,1,0,0,0,0,1,-1,-1
3,1,1,1,1,0,0,0,-1,-1,-1,-1
4,1,1,1,1,0,0,-1,0,-1,-1,-1


In [4]:
X = df[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'player']].values
y = df['move'].values

In [5]:
print(X[:7])

[[ 1  1  1  1  0  0  1  0  0 -1]
 [ 1  1  1  1  0  0  0  1  0 -1]
 [ 1  1  1  1  0  0  0  0  1 -1]
 [ 1  1  1  1  0  0  0 -1 -1 -1]
 [ 1  1  1  1  0  0 -1  0 -1 -1]
 [ 1  1  1  1  0  0 -1 -1  0 -1]
 [ 1  1  1  1  0 -1  0  0 -1 -1]]


In [6]:
print(y[:7])

[-1 -1 -1 -1 -1 -1 -1]


In [7]:
y_encoded = np.zeros([len(y), 10])

for i in range(0, len(y)):
    if(y[i] == -1):
        y_encoded[i][0] = 1.0
    else:
        y_encoded[i][int(y[i])] = 1.0

In [8]:
y_encoded[:7]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.float32)

In [10]:
X_tensor.shape, y_tensor.shape

(torch.Size([5478, 10]), torch.Size([5478, 10]))

In [11]:
#from sklearn.model_selection import train_test_split

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.1, random_state=42)

In [13]:
#print(len(X_test))

In [14]:
"""class TicTacToeTrainer(nn.Module):

    def __init__(self, input_size=10, hidden_size=128, output_size=10):
        super(TicTacToeTrainer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.5)   # 50% dropout

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        return self.fc3(x)"""


'class TicTacToeTrainer(nn.Module):\n\n    def __init__(self, input_size=10, hidden_size=128, output_size=10):\n        super(TicTacToeTrainer, self).__init__()\n        self.fc1 = nn.Linear(input_size, hidden_size)\n        self.bn1 = nn.BatchNorm1d(hidden_size)\n        self.dropout1 = nn.Dropout(0.5)   # 50% dropout\n\n        self.fc2 = nn.Linear(hidden_size, hidden_size)\n        self.bn2 = nn.BatchNorm1d(hidden_size)\n        self.dropout2 = nn.Dropout(0.5)\n\n        self.fc3 = nn.Linear(hidden_size, output_size)\n        self.relu = nn.ReLU()\n\n\n    def forward(self, x):\n        x = self.relu(self.bn1(self.fc1(x)))\n        x = self.dropout1(x)\n        x = self.relu(self.bn2(self.fc2(x)))\n        x = self.dropout2(x)\n        return self.fc3(x)'

In [15]:
class TicTacToeTrainer(nn.Module):

    def __init__(self):
        super(TicTacToeTrainer, self).__init__()

        self.fc1 = nn.Linear(10, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10) #logits as output

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)

        return x
       

In [16]:
SNet = TicTacToeTrainer()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SNet = SNet.to(device)

In [18]:
#X_train = X_train.to(device)
#y_train = y_train.to(device)
#X_test = X_test.to(device)
#y_test = y_test.to(device)
X_train = X_tensor.to(device)
y_train = y_tensor.to(device)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer =  torch.optim.Adam(SNet.parameters(), lr=1e-3, weight_decay=1e-5)

In [20]:
num_epochs = 10000 # 1 epoch, 1 traversal through the training set
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SNet = SNet.to(device)
SNet.train()
for epoch in range(num_epochs):
    optimizer.zero_grad() #zero out the gradients
    outputs = SNet(X_train) #run the forward pass
    loss = criterion(outputs, y_train) # compute the loss
    loss.backward() #gradient calculation happens from here
    optimizer.step() #takes the step based on the gradients and lr

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.6f}')

Epoch [10/10000], Loss: 2.269651
Epoch [20/10000], Loss: 2.217851
Epoch [30/10000], Loss: 2.136278
Epoch [40/10000], Loss: 2.015136
Epoch [50/10000], Loss: 1.870151
Epoch [60/10000], Loss: 1.721081
Epoch [70/10000], Loss: 1.584432
Epoch [80/10000], Loss: 1.464432
Epoch [90/10000], Loss: 1.355095
Epoch [100/10000], Loss: 1.259590
Epoch [110/10000], Loss: 1.177744
Epoch [120/10000], Loss: 1.106914
Epoch [130/10000], Loss: 1.049495
Epoch [140/10000], Loss: 1.007549
Epoch [150/10000], Loss: 0.975959
Epoch [160/10000], Loss: 0.948550
Epoch [170/10000], Loss: 0.923673
Epoch [180/10000], Loss: 0.901376
Epoch [190/10000], Loss: 0.880226
Epoch [200/10000], Loss: 0.858440
Epoch [210/10000], Loss: 0.835407
Epoch [220/10000], Loss: 0.812607
Epoch [230/10000], Loss: 0.792878
Epoch [240/10000], Loss: 0.776143
Epoch [250/10000], Loss: 0.761246
Epoch [260/10000], Loss: 0.747837
Epoch [270/10000], Loss: 0.736103
Epoch [280/10000], Loss: 0.726008
Epoch [290/10000], Loss: 0.717188
Epoch [300/10000], Loss

In [21]:
SNet.eval()
with torch.no_grad():
    outputs = SNet(X_test)
    _, predicted = torch.max(outputs.data, 1)
    _, actual = torch.max(y_test.data, 1)

    accuracy = (predicted == actual).sum().item()/actual.size(0)*100.0

    print(f'Test Accuracy : {accuracy}')

NameError: name 'X_test' is not defined

In [22]:
SNet.eval()
with torch.no_grad():
    outputs = SNet(X_train)
    _, predicted = torch.max(outputs.data, 1)
    _, actual = torch.max(y_train.data, 1)

    accuracy = (predicted == actual).sum().item()/actual.size(0)*100.0

    print(f'Training Accuracy : {accuracy}')

Training Accuracy : 100.0


In [23]:
torch.save(SNet.state_dict(), "TicTacToe_Weights.pth")